In [3]:
from collections import defaultdict, deque, Counter
import numpy as np
from tqdm import tqdm_notebook as tqdm 
import pandas as pd

#from sklearn.preprocessing import normalize

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix

In [4]:
# dummy maxmimum position variable. assign the position of blocks that 
# will never get accessed a value greater than this value. This way OPT
# can be fooled to think that the block will be accessed but at a position
# far-far-away in time.
infinite_index = 1000000000 
sample_interval = 1000
num_params = 3
eviction = 100

In [5]:
filename = "cheetah.cs.fiu.edu-110108-113008.1.blkparse"
df = pd.read_csv(filename, sep=' ',header = None)
df.columns = ['timestamp','pid','pname','blockNo', \
              'blockSize', 'readOrWrite', 'bdMajor', 'bdMinor', 'hash']
blocktrace = df['blockNo'].tolist()
len(blocktrace)

1322890

In [6]:
lruCorrect = 0
lruIncorrect = 0

def lruPredict(C,LRUQ,Y_OPT):
    global lruCorrect, lruIncorrect
    Y_current = []
    KV = defaultdict(int)
    for i in range(len(LRUQ)):
        KV[LRUQ[i]] = len(LRUQ) - i
    KV_sorted = Counter(KV)
    evict_dict = dict(KV_sorted.most_common(eviction))
    for e in C:
        if e in evict_dict:
            Y_current.append(1)
        else:
            Y_current.append(0)
    for i in range(len(Y_current)):
        if Y_current[i] is Y_OPT[i]:
            lruCorrect+=1
        else:
            lruIncorrect+=1
    return Y_current

'''
    given C, use LFUDict to find eviction number of blocks from the Cache
    compare it with Y_OPT and store number of places the two differ
    The number of correct and incorrect predictions with respect to OPT.
'''

lfuCorrect = 0
lfuIncorrect = 0

def lfuPredict(C,LFUDict,Y_OPT):
    global lfuCorrect, lfuIncorrect
    Y_current = []
    KV = defaultdict()
    for e in C:
        KV[e] = LFUDict[e]
    KV_sorted = Counter(KV)
    evict_dict = dict(KV_sorted.most_common(eviction))
    for e in C:
        if e in evict_dict:
            Y_current.append(1)
        else:
            Y_current.append(0)
    for i in range(len(Y_current)):
        if Y_current[i] is Y_OPT[i]:
            lfuCorrect+=1
        else:
            lfuIncorrect+=1
    return Y_current

# return "eviction" blocks that are being accessed furthest
# from the cache that was sent to us.

def getY(C,OPT):
    global infinite_index
    KV = defaultdict(int)
    Y_current = []
    for e in C:
        if len(OPT[e]) is not 0:
            KV[e] = OPT[e][0]
        else:
            KV[e] = infinite_index
            infinite_index+=1
    # extract "eviction" blocks from KV_sorted hashmap
    KV_sorted = Counter(KV)
    evict_dict = dict(KV_sorted.most_common(eviction))
    for e in C:
        if e in evict_dict:
            Y_current.append(1)
        else:
            Y_current.append(0)
    return Y_current

def getLFURow(LFUDict, C):
    x_lfurow = []
    for e in C:
        x_lfurow.append(LFUDict[e])
    norm = x_lfurow / np.linalg.norm(x_lfurow)
    return norm
    
def getLRURow(LRUQ, C):
    x_lrurow = []
    KV = defaultdict(int)
    for i in range(len(LRUQ)):
        KV[LRUQ[i]] = len(C) - i
    for e in C:
        x_lrurow.append(KV[e])
    norm = x_lrurow / np.linalg.norm(x_lrurow)
    return norm

def getX(LRUQ, LFUDict, C):
    X_lfurow = getLFURow(LFUDict, C)
    X_lrurow = getLRURow(LRUQ, C)
    X_bno    = C / np.linalg.norm(C)
    return (np.column_stack((X_lfurow, X_lrurow, X_bno)))

# appends OPT sample to X, Y arrays

X = np.array([], dtype=np.int64).reshape(0,num_params)
Y = np.array([], dtype=np.int64).reshape(0,1)

# C - cache, LFUDict - dictionary containing block-> access frequency
# LRUQ - order of element access in Cache.

def populateData(frequency, recency, Cache, block_index):
    '''
    Input: 
    Output:
    '''
    global X,Y
    Cache = list(Cache)
    Y_current = getY(Cache, block_index)
    X_current = getX(recency, frequency, Cache)

    Y = np.append(Y, Y_current)
    X = np.concatenate((X,X_current))
    return Y_current


In [7]:
def belady_opt(blocktrace, frame):
    '''
    INPUT
    ==========
    blocktrace = list of block request sequence
    frame = size of the cache
    
    OUTPUT
    ==========
    hitrate 
    '''
    global sample_interval # interval of choice for sampling
    global infinite_index # should be a large integer
    
    block_index = defaultdict(deque) 
    # dictionary with block number as key and list
    # of index value in blocktrace
    
    upcoming_index = defaultdict(int)
    # dictionary with index number as key and value as block
    
    frequency = defaultdict(int)
    # dictionary of block as key and number
    # of times it's been requested so far
    
    recency = list()
    # list of block in order of their request
    
    Cache = set()
    # Cache with block
    
    hit, miss = 0, 0
    
    # populate the block_index
    for i, block in enumerate(tqdm(blocktrace, \
                              desc="buidling index", leave=False)):
        block_index[block].append(i)
        
    # sequential block requests start
    for i, block in enumerate(tqdm(blocktrace, desc="sequence", leave=False)):
        
        # increament the frequency number for the block
        frequency[block] += 1
        
        # make sure block has the value in block_index dictionary 
        # as current seq_number
        if len(block_index[block]) != 0 and block_index[block][0] == i:
            
            # if yes, remove the first element of block_index[block]
            block_index[block].popleft()
        
        # if block exist in current cache
        if block in Cache:
            
            # increment hit
            hit += 1
            
            # update the recency
            recency.remove(block)
            recency.append(block)
            
            # update upcoming_index
            if i in upcoming_index:
                
                # delete old index
                del upcoming_index[i]
        
                if len(block_index[block]) is not 0:
                    # add new upcoming index
                    upcoming_index[block_index[block][0]] = block
                    # remove index from block_index
                    block_index[block].popleft()
                else:
                    # add a large integer as index
                    upcoming_index[infinite_index] = block
                    # increament large integer
                    infinite_index+=1
           
        # block not in current cache
        else:
            
            # increament miss
            miss += 1
            
            # if cache has no free space
            if len(Cache) == frame:
                
                # evict the farthest block in future request from cache
                if len(upcoming_index) != 0:
                    
                    # find the farthest i.e. max_index in upcoming_index
                    max_index = max(upcoming_index)
                    
                    # remove the block with max_index from cache
                    Cache.remove(upcoming_index[max_index])
                    
                    # remove the block with max_index from recency dict
                    recency.remove(upcoming_index[max_index])
                    
                    # remove max_index element from upcoming_index
                    del upcoming_index[max_index]
                    
            # add upcoming request of current block in upcoming_index
            if len(block_index[block]) != 0:
                
                # add upcoming index of block
                upcoming_index[block_index[block][0]] = block
               
                # remove the index from block_index 
                block_index[block].popleft()
            
            else:
                
                # add a large integer as index
                upcoming_index[infinite_index] = block
                
                # increament high number
                infinite_index += 1
                
            
            # add block into Cache
            Cache.add(block)
            
            # add block into recency
            recency.append(block)
            
            #### regression extra part
            if (i % sample_interval +1 == sample_interval):
                Y_OPT = populateData(frequency, recency, Cache, block_index)
                lruPredict(Cache,recency,Y_OPT)
                lfuPredict(Cache,frequency,Y_OPT)
            
    # calculate hitrate
    hitrate = hit / (hit + miss)

    return hitrate

In [8]:
belady_opt(blocktrace, 500)

0.06331214235499551

In [ ]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y ,test_size=0.3, random_state=0,shuffle=False)

#Fitting Logistic Regression Model
logreg = LogisticRegression()
logreg.fit(X_train, Y_train)

Y_pred = logreg.predict(X_test)

print('Accuracy of logistic regression classifier on test set: {:.2f}'.format(logreg.score(X_test, Y_test)))

confusion_matrix = confusion_matrix(Y_test,Y_pred)
print (confusion_matrix)

print ("LFU Correct / Incorrect Ratio")
total = lfuCorrect + lfuIncorrect
print ( (total - (lfuIncorrect/2) ) / total )

print ("LRU Correct / Incorrect Ratio")
total = lruCorrect + lruIncorrect
print ( (total - (lruIncorrect/2) ) / total )

In [ ]:
def getX2(LRUQ, LFUDict, C):
    C = list(C)
    X_lfurow = getLFURow(LFUDict, C)
    X_lrurow = getLRURow(LRUQ, C)
    X_bno    = C / np.linalg.norm(C)
    oldreturn = np.column_stack((X_lfurow, X_lrurow, X_bno))
    predprob =  logreg.predict_proba(oldreturn)[:,1]
    index  = np.argmax(predprob)
    return C[index]

In [ ]:
cache_size=500
accuracy=[]
for i in range(int(len(X_test)/cache_size)):
    Y_pred = logreg.predict_proba(X_test[i*cache_size:(i+1)*cache_size])
    ones = np.count_nonzero(Y_test[i*cache_size:(i+1)*cache_size])
    Y_binary = np.zeros(Y_pred.shape[0])
    ind = np.argpartition(Y_pred[:,1], -ones)[-ones:]
    Y_binary[ind] = 1
    if ones != 0:
        s = Y_binary * Y_test[i*cache_size:(i+1)*cache_size]
        accuracy.append(np.count_nonzero(s) / ones)
np.array(accuracy).mean()

In [ ]:
def ourcache(blocktrace,frame):
    '''
    INPUT
    ==========
    blocktrace = list of block request sequence
    frame = size of the cache
    
    OUTPUT
    ==========
    hitrate 
    '''
    #global sample_interval # interval of choice for sampling
    global infinite_index # should be a large integer
    
    #block_index = defaultdict(deque) 
    # dictionary with block number as key and list
    # of index value in blocktrace
    
    #upcoming_index = defaultdict(int)
    # dictionary with index number as key and value as block
    
    frequency = defaultdict(int)
    # dictionary of block as key and number
    # of times it's been requested so far
    
    recency = list()
    # list of block in order of their request
    
    Cache = deque()
    # Cache with block
    
    hit, miss = 0, 0
    
    # populate the block_index
    #for i, block in enumerate(tqdm(blocktrace, \
      #                        desc="buidling index", leave=False)):
     #   block_index[block].append(i)
        
    # sequential block requests start
    for i, block in enumerate(tqdm(blocktrace, desc="sequence", leave=False)):
        
        # increament the frequency number for the block
        frequency[block] += 1
        
        # make sure block has the value in block_index dictionary 
        # as current seq_number
        #if len(block_index[block]) != 0 and block_index[block][0] == i:
            
            # if yes, remove the first element of block_index[block]
        #    block_index[block].popleft()
        
        # if block exist in current cache
        if block in Cache:
            
            # increment hit
            hit += 1
            
            # update the recency
            recency.remove(block)
            recency.append(block)
            
            # update upcoming_index
            #if i in upcoming_index:
                
                # delete old index
             #   del upcoming_index[i]
        
              #  if len(block_index[block]) is not 0:
                    # add new upcoming index
               #     upcoming_index[block_index[block][0]] = block
                    # remove index from block_index
                #    block_index[block].popleft()
#                 else:
#                     # add a large integer as index
#                     upcoming_index[infinite_index] = block
#                     # increament large integer
#                     infinite_index+=1
           
        # block not in current cache
        else:
            
            # increament miss
            miss += 1
            
            # if cache has no free space
            if len(Cache) == frame:
                

                X_current = getX2(recency, frequency, Cache)

                
                Cache.remove(X_current)
                
                # evict the farthest block in future request from cache
#                 if len(upcoming_index) != 0:
                    
#                     # find the farthest i.e. max_index in upcoming_index
#                     max_index = max(upcoming_index)
                    
#                     # remove the block with max_index from cache
#                     Cache.remove(upcoming_index[max_index])
                    
#                     # remove the block with max_index from recency dict
#                     recency.remove(upcoming_index[max_index])
                    
#                     # remove max_index element from upcoming_index
#                     del upcoming_index[max_index]
                    
            # add upcoming request of current block in upcoming_index
#             if len(block_index[block]) != 0:
                
#                 # add upcoming index of block
#                 upcoming_index[block_index[block][0]] = block
               
#                 # remove the index from block_index 
#                 block_index[block].popleft()
            
#             else:
                
#                 # add a large integer as index
#                 upcoming_index[infinite_index] = block
                
#                 # increament high number
#                 infinite_index += 1
                
            
            # add block into Cache
            Cache.append(block)
            
            # add block into recency
            recency.append(block)
            
            #### regression extra part
#             if (i % sample_interval +1 == sample_interval):
#                 Y_OPT = populateData(frequency, recency, Cache, block_index)
#                 lruPredict(Cache,recency,Y_OPT)
#                 lfuPredict(Cache,frequency,Y_OPT)
            
    # calculate hitrate
    hitrate = hit / (hit + miss)

    return hitrate

In [ ]:
ourcache(blocktrace, 3)

In [ ]:
def LRU(blocktrace, frame):
    
    cache = set()
    recency = deque()
    hit, miss = 0, 0
    
    for block in tqdm(blocktrace, leave=False):
        
        if block in cache:
            recency.remove(block)
            recency.append(block)
            hit += 1
            
        elif len(cache) < frame:
            cache.add(block)
            recency.append(block)
            miss += 1
            
        else:
            cache.remove(recency[0])
            recency.popleft()
            cache.add(block)
            recency.append(block)
            miss += 1
    
    hitrate = hit / (hit + miss)
    return hitrate

In [ ]:
LRU(blocktrace, 3)

# Training on whole day and testing on the same day trace of the next week

In [9]:
logreg = LogisticRegression()
logreg.fit(X, Y)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

In [22]:
filename = "cheetah.cs.fiu.edu-110108-113008.8.blkparse"
df = pd.read_csv(filename, sep=' ',header = None)
df.columns = ['timestamp','pid','pname','blockNo', \
              'blockSize', 'readOrWrite', 'bdMajor', 'bdMinor', 'hash']
blocktrace = df['blockNo'].tolist()[:40000]
len(blocktrace)

40000

In [23]:
def LRU(blocktrace, frame):
    
    cache = set()
    recency = deque()
    hit, miss = 0, 0
    
    for block in tqdm(blocktrace, leave=False):
        
        if block in cache:
            recency.remove(block)
            recency.append(block)
            hit += 1
            
        elif len(cache) < frame:
            cache.add(block)
            recency.append(block)
            miss += 1
            
        else:
            cache.remove(recency[0])
            recency.popleft()
            cache.add(block)
            recency.append(block)
            miss += 1
    
    hitrate = hit / (hit + miss)
    return hitrate

In [24]:
LRU(blocktrace, 500)

0.2528

In [25]:
def getX2(LRUQ, LFUDict, C):
    C = list(C)
    X_lfurow = getLFURow(LFUDict, C)
    X_lrurow = getLRURow(LRUQ, C)
    X_bno    = C / np.linalg.norm(C)
    oldreturn = np.column_stack((X_lfurow, X_lrurow, X_bno))
    predprob =  logreg.predict_proba(oldreturn)[:,1]
    index  = np.argmax(predprob)
    return C[index]

In [26]:
def ourcache(blocktrace,frame):
    '''
    INPUT
    ==========
    blocktrace = list of block request sequence
    frame = size of the cache
    
    OUTPUT
    ==========
    hitrate 
    '''
    #global sample_interval # interval of choice for sampling
    global infinite_index # should be a large integer
    
    #block_index = defaultdict(deque) 
    # dictionary with block number as key and list
    # of index value in blocktrace
    
    #upcoming_index = defaultdict(int)
    # dictionary with index number as key and value as block
    
    frequency = defaultdict(int)
    # dictionary of block as key and number
    # of times it's been requested so far
    
    recency = list()
    # list of block in order of their request
    
    Cache = deque()
    # Cache with block
    
    hit, miss = 0, 0
    
    # populate the block_index
    #for i, block in enumerate(tqdm(blocktrace, \
      #                        desc="buidling index", leave=False)):
     #   block_index[block].append(i)
        
    # sequential block requests start
    for i, block in enumerate(tqdm(blocktrace, desc="sequence", leave=False)):
        
        # increament the frequency number for the block
        frequency[block] += 1
        
        # make sure block has the value in block_index dictionary 
        # as current seq_number
        #if len(block_index[block]) != 0 and block_index[block][0] == i:
            
            # if yes, remove the first element of block_index[block]
        #    block_index[block].popleft()
        
        # if block exist in current cache
        if block in Cache:
            
            # increment hit
            hit += 1
            
            # update the recency
            recency.remove(block)
            recency.append(block)
            
            # update upcoming_index
            #if i in upcoming_index:
                
                # delete old index
             #   del upcoming_index[i]
        
              #  if len(block_index[block]) is not 0:
                    # add new upcoming index
               #     upcoming_index[block_index[block][0]] = block
                    # remove index from block_index
                #    block_index[block].popleft()
#                 else:
#                     # add a large integer as index
#                     upcoming_index[infinite_index] = block
#                     # increament large integer
#                     infinite_index+=1
           
        # block not in current cache
        else:
            
            # increament miss
            miss += 1
            
            # if cache has no free space
            if len(Cache) == frame:
                

                X_current = getX2(recency, frequency, Cache)

                
                Cache.remove(X_current)
                
                # evict the farthest block in future request from cache
#                 if len(upcoming_index) != 0:
                    
#                     # find the farthest i.e. max_index in upcoming_index
#                     max_index = max(upcoming_index)
                    
#                     # remove the block with max_index from cache
#                     Cache.remove(upcoming_index[max_index])
                    
#                     # remove the block with max_index from recency dict
#                     recency.remove(upcoming_index[max_index])
                    
#                     # remove max_index element from upcoming_index
#                     del upcoming_index[max_index]
                    
            # add upcoming request of current block in upcoming_index
#             if len(block_index[block]) != 0:
                
#                 # add upcoming index of block
#                 upcoming_index[block_index[block][0]] = block
               
#                 # remove the index from block_index 
#                 block_index[block].popleft()
            
#             else:
                
#                 # add a large integer as index
#                 upcoming_index[infinite_index] = block
                
#                 # increament high number
#                 infinite_index += 1
                
            
            # add block into Cache
            Cache.append(block)
            
            # add block into recency
            recency.append(block)
            
            #### regression extra part
#             if (i % sample_interval +1 == sample_interval):
#                 Y_OPT = populateData(frequency, recency, Cache, block_index)
#                 lruPredict(Cache,recency,Y_OPT)
#                 lfuPredict(Cache,frequency,Y_OPT)
            
    # calculate hitrate
    hitrate = hit / (hit + miss)

    return hitrate

In [27]:
ourcache(blocktrace, 500)

0.2526